# Monte Carlo Demonstration

In this lecture, we will demonstrate Monte Carlo methods. This is a useful tool in testing our estimation techniques. Recall that omitting relevant variables in OLS can sometimes lead to biased regression estimates. To see this in action, we will simulate data and run these regressions.  

## Theoretical Motivation

Recall the example of calculating Pi by throwing darts at a board.

**Discussion:** Describe the procedure that would be used to calculate Pi. Why does this work?

<img src="Pi_30K.gif"/>

**Discussion:** 

  * Let each dart throw within the circle be a 1. Otherwise, let it be a zero. What is the expected value of this random variable? How would you write this as an integral?
  * How can we extend this principle to arbitrary integrals?
  

  
**Monte Carlo Integration**

$\newcommand{\dd}{\, \mathrm{d}}$
Suppose we have a function $f$ and we want to evaluate the definite integral $\int_\Omega f(x) \dd x$. Monte Carlo integration works on the principle that we can rewrite an integral as the expected value of some random variable. We can then simulate random numbers from a distribution and use the law of large numbers to argue that the sample mean will converge to the population mean. For example, suppose we have some random variable $X$ that has support $\Omega$ (that is $X \in \Omega$). Let the probability density function (PDF) of this random variable be $\pi$, so that $\int_\Omega \pi(x) \dd x = 1$. Define a function $g$ such that $g(x) = f(x)/\pi(x)$. Then

$$
E[g(X)] = E\left[ \frac{f(X)}{\pi(X)}\right] = \int_\Omega \frac{f(x)}{\pi(x)} \pi(x) \dd x = \int_\Omega f(x) \dd x.
$$

Now, if we can simulate draws of the random variable $X$, the law of large numbers tells us that

$$
\text{plim}_{N\rightarrow \infty} \frac{1}{N} \sum_{i=1}^N g(X_i) = E[g(X)] = \int_\Omega f(x) \dd x.
$$

The choice of distribution for $X$ gives us some flexibility in approximating this integral. Some distributions may have better performance than others (in terms of accuracy with respect to sample size). In the most basic case, we will choose a uniform distribution, so that $\pi(x) = c$. In this case,

$$
1 = \int_\Omega \pi(x) \dd x = \int_\Omega c \dd x = c V,
$$

where $V$ is the volume of the space $\Omega$
$$
V = \int_\Omega \, \mathrm d x .
$$ This implies that our integral can be approximated
$$
\text{plim}_{N\rightarrow \infty} V \frac{1}{N} \sum_{i=1}^N f(X_i) = \int_\Omega f(x) \dd x,
$$
where $X$ is uniformly distributed over $\Omega$.
This is the formula that you will find on the related Wikipedia article. This topic can be explored further [there.](https://en.wikipedia.org/wiki/Monte_Carlo_integration) Monte Carlo simulation, in general, is a tool to explore properties of distributions that are otherwise hard to calculate.

In [ ]:
# !pip install linearmodels
# import linearmodels

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats
import statsmodels.formula.api as smf
import statsmodels.api as sm
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()

Let's integrate a half circle using Monte Carlo integration.
$$
y^2 + x^2 = 1
$$
so
$$
y = \sqrt[+]{1 - x^2}
$$

In [ ]:
N = 100
x = np.random.uniform(low=-1.0, high=1.0, size=N)
y = np.sqrt(1 - x**2) 
V = 2

In [ ]:
plt.plot(x,y, '.')

In [ ]:
np.mean(y) * V

In [ ]:
N = 10000
x = np.random.uniform(low=-1.0, high=1.0, size=N)
y = np.sqrt(1 - x**2) 
V = 2
print('Approximation = ', np.mean(y) * V)
print('Built-in value = ', np.pi / 2)

## Simple Example and Simulation

Consider a common source of endogeneity in simple linear models: ommited variables. Suppose the data generating process is given by

$$
y = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \epsilon,
$$
where $\epsilon$ is independent of $x_1$ and $x_2$.

If we estimate the equation $y = \tilde \beta_0 + \tilde \beta_1 x_1 + \nu$, where $\nu = \beta_2 x_2 + \epsilon$, then it may occur that our estimates of $\tilde \beta_1$ will be biased?

**Discussion: When will our estimate $\tilde \beta_1$ be biased? When will it not be biased?**

We can run a quick experiment to find out. Let $x_1$ and $x_2$ be positively correlated and let $\beta_2$ be equal to 3. Will $\tilde \beta_1$ over- or under-estimate $\beta_1$?

In [ ]:
def simulate_data_ex1(N=50, seed=65594):
    if seed == None:
        pass
    else:
        np.random.seed(seed)
    # Let's ensure that x_1 and x_2 are correlated
    cov = np.array([[2,1],
                    [1,2]])
    mean = np.array([0, 0])
    X = scipy.stats.multivariate_normal.rvs(mean=mean, cov=cov, size=N)
    epsilon = np.random.randn(N,1)
    beta0, beta1, beta2 = 0, 1, 1
    df = pd.DataFrame(X, columns=['x1', 'x2'])
    df['epsilon'] = epsilon
    df['y'] = beta0 + beta1 * df.x1 + beta2 * df.x2 + df.epsilon
    df = df[['y', 'x1', 'x2', 'epsilon']]
    return df

In [ ]:
df = simulate_data_ex1()
df.head()

In [ ]:
# Using classic API (need to create matrices ourselves, including adding the constant)
endog = df.y
exog = sm.add_constant(df.x1)
reg = sm.OLS(endog=endog, exog=exog).fit()
reg.summary()

In [ ]:
exog.head()

In [ ]:
# Use formula API
reg = smf.ols('y ~ x1', df).fit()
reg.summary()

**Discussion: Notice that in this result we underestimated $\beta_1$. Is this what you expected? Why?**

Let's try it again.

In [ ]:
df = simulate_data_ex1(N=50, seed=10954)
reg = smf.ols('y ~ x1', df).fit()
reg.summary()

Why is this happening?

In [ ]:
df.head()

In [ ]:
df.cov()

In [ ]:
sns.jointplot(x=df.x1, y=df.epsilon, kind="hex");

In [ ]:
sns.jointplot(x=df.x1, y=df.x2, kind="hex");

One more try. **Do you notice anything strange?**

In [ ]:
df = simulate_data_ex1(N=50, seed=87548)
reg = smf.ols('y ~ x1', df).fit()
reg.summary()

Now this:

In [ ]:
df = simulate_data_ex1(N=50, seed=75100)
reg = smf.ols('y ~ x1', df).fit()
reg.summary()

In [ ]:
df = simulate_data_ex1(N=50, seed=100)
reg = smf.ols('y ~ x1', df).fit()
reg.summary()

These results are occuring simply by chance. In fact, I simulated data many times, looping through various "seeds", until I found the results that I wanted. See the code below.

In [ ]:
# beta0, beta1, beta2 = 0, 1, 1
# M = 1000
# biases = np.zeros(M)
# for i in range(M):
#     df = simulate_data_ex1(N=50, seed=i)
#     reg = smf.ols('y ~ x1', df).fit()
#     biases[i] = reg.params.x1 - beta1

# # # Run the above code with M = 100_000, N=50, betas = 0,1,1
# # # np.savetxt('biases_N50_beta011_M100000.txt', biases)

In [ ]:
# ind_min = np.argmin(biases)
# biases[ind_min]

In [ ]:
# ind_max = np.argmax(biases)
# biases[ind_max]

In [ ]:
# ind_amin = np.argmin(np.abs(biases))
# biases[ind_amin]

In [ ]:
# ind_amax = np.argmax(np.abs(biases))
# biases[ind_amax]

In [ ]:
# print(ind_min, ind_max, ind_amin, ind_amax)

### Omitted Variable Bias

Recall that in the (probability) limit, 

$$
\tilde \beta_1^N \overset{p}{\rightarrow} \frac{\text{Cov}(y, x_1)}{\text{Var}(x_1)} = \frac{\text{Cov}(\beta_1 x_1 + \beta_2 x_2, x_1)}{\text{Var}(x_1)} = \beta_1 + \beta_2  \frac{\text{Cov}(x_2, x_1)}{\text{Var}(x_1)}
$$

We can run a simulation to verify this formula.


## Monte Carlo Method

We will do `M=1000` Monte-Carlo loops. For each loop, we will create a new dataset with `N=50` data points and then we will estimate the parameters. We will record the parameter estimates from each run. We will analyze the properties of these records.

In [ ]:
reg.params.index

In [ ]:
M = 10
results = pd.DataFrame(index=range(M), columns=reg.params.index)
results.head()

In [ ]:
results.loc[0, :] = reg.params
results.head()

In [ ]:
np.random.seed(100)
M = 1000
results = pd.DataFrame(index=range(M), columns=reg.params.index, dtype=np.float)
for m in range(M):
    # Set `seed` parameter to `None` so that I don't reset
    # the seed every time.
    df = simulate_data_ex1(seed=None)
    reg = smf.ols('y ~ x1', df).fit()
    results.loc[m, :] = reg.params

In [ ]:
sns.distplot(results.x1)

In [ ]:
results.hist();

In [ ]:
results.describe()

Now let's compare this to the theory:

$$
\tilde \beta^N_1 \overset{p}{\rightarrow}  \beta_1 + \beta_2  \frac{\text{Cov}(x_2, x_1)}{\text{Var}(x_1)}
$$

We can look above to see how we defined the parameters of our simulation and calculate these quantities.

In [ ]:
beta0, beta1, beta2 = 0, 1, 1
c = 1
v = 2
tilde_beta1 = beta1 + beta2 * c/v
tilde_beta1

In [ ]:
np.random.seed(100)
M = 1000
N = 500
results2 = pd.DataFrame(index=range(M), columns=reg.params.index, dtype=np.float)
for m in range(M):
    # Set `seed` parameter to `None` so that I don't reset
    # the seed every time.
    df = simulate_data_ex1(N=N, seed=None)
    reg = smf.ols('y ~ x1', df).fit()
    results2.loc[m, :] = reg.params

In [ ]:
results2.describe()

In [ ]:
sns.distplot(results.x1, label='$N=50$')
sns.distplot(results2.x1, label='$N=500$')
plt.legend();

In [ ]:
np.random.seed(100)
M = 1000
N = 5000
results3 = pd.DataFrame(index=range(M), columns=reg.params.index, dtype=np.float)
for m in range(M):
    # Set `seed` parameter to `None` so that I don't reset
    # the seed every time.
    df = simulate_data_ex1(N=N, seed=None)
    reg = smf.ols('y ~ x1', df).fit()
    results3.loc[m, :] = reg.params

In [ ]:
sns.distplot(results.x1, label='$N=50$')
sns.distplot(results2.x1, label='$N=500$')
sns.distplot(results3.x1, label='$N=5000$')
plt.legend();